# Generate Synthetic Datasets

This notebook will create the synthetic dataset in our experiments.

In [1]:
import numpy as np
import pandas as pd

## Motivating Dataset

Motivating Dataset with 4 circles that move in a circular path with different area changes.

In [2]:
timestamps = 20
number_objects = 4
number_turns = 1

def increasing_radius(a, b):
    """Linear function to represent an increasing radius"""
    return lambda t : a * t + b
    
def decreasing_radius(a, b):
    """Linear function to represent a decreasing radius"""
    return lambda t : max(0.05, - a * t + b)

def constant_radius(a):
    """Constant function to represent a constant radius"""
    return lambda t : a

objects_radius = [constant_radius(0.1), increasing_radius(0.4, 0.05), 
                  increasing_radius(0.4, 0.05), decreasing_radius(0.29, 0.3), ]

def orbit_radius_func(t, i):
    """Function that return the radius of the orbit of the object i at time t"""
    if i == 0:
        return 0.1
    elif i == 1:
        if t < timestamps/2:
            return 0.5
        else:
            return (t+1-timestamps/2)/(timestamps/2) * 0.15 + 0.5
    elif i == 2:
        return 0.8
    elif i == 3:
        return 1
    
objects_orbit_radius = [0.1, 0.5, 0.8, 1]

direction = [1, 1, 1, -1]

# For each timestamp, for each object, the values: circle xcenter, circle ycenter, circle radius
circle_objects = np.zeros((timestamps, number_objects, 3))
for t in range(timestamps):
    for i in range(number_objects):
        d = direction[i]
        orbit_radius = orbit_radius_func(t, i)
        xcenter = np.cos(2*np.pi*t/timestamps * d * number_turns) * orbit_radius
        ycenter = np.sin(2*np.pi*t/timestamps * d * number_turns) * orbit_radius
        radius = objects_radius[i](t/timestamps)
        circle_objects[t, i, :] = [xcenter, ycenter, radius]
        
# Transform to DataFrame
circle_objects_df = {"timestep": [], "xcenter": [], "ycenter":[], "radius": [], "object": [], "points": []}
for t in range(timestamps):
    for i in range(number_objects):
        circle_objects_df["timestep"].append(t)
        circle_objects_df["object"].append(i)
        circle_objects_df["xcenter"].append(circle_objects[t, i, 0])
        circle_objects_df["ycenter"].append(circle_objects[t, i, 1])
        circle_objects_df["radius"].append(circle_objects[t, i, 2])
        
        angle_samples = np.linspace(0, 2*np.pi, 20)
        points = []
        for theta in angle_samples:
            points.append([circle_objects[t, i, 0] + circle_objects[t, i, 2] * np.cos(theta),
                          circle_objects[t, i, 1] + circle_objects[t, i, 2] * np.sin(theta)])
        circle_objects_df["points"].append(points)
        
circle_objects_df = pd.DataFrame(circle_objects_df)
circle_objects_df["area"] = circle_objects_df["radius"]*circle_objects_df["radius"] *np.pi

colormap = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
circle_objects_df["color"] = circle_objects_df["object"].apply(lambda x: colormap[x])

circle_objects_df.to_csv("../data/processed/motivating.csv", index = False)